In [1]:
#import everything here
import numpy as np
import pandas as pd
import quandl

Calculating the Simple moving average(SMA) of an equity price over a period (Middle Band)
Upper/Lower Bands = 2 S.D. from the middle band

In [3]:
#API data import
quandl.ApiConfig.api_key = "fYkH3s6N_8cAWa98E-JH"  #To ensure that my requests to Quandl are authenticated
API_key = "fYkH3s6N_8cAWa98E-JH"  #will be imported to git.ignore later
#curl "https://www.quandl.com/api/v3/datasets/WIKI/FB.json?column_index=4&start_date=2014-01-01&end_date=2014-12-31&collapse=monthly&transform=rdiff&api_key=YOURAPIKEY"

meta_data = pd.read_csv('BSE_metadata.csv') #read meta data so we can understand the API request convention more easily
print(meta_data)

           code                                              name  \
0     BOM500002                      ABB India Limited EOD Prices   
1     BOM500003                   AEGIS LOGISTICS LTD. EOD Prices   
2     BOM500008              AMARA RAJA BATTERIES LTD. EOD Prices   
3     BOM500009      AMBALAL SARABHAI ENTERPRISES LTD. EOD Prices   
4     BOM500010  HOUSING DEVELOPMENT FINANCE CORP.LTD. EOD Prices   
...         ...                                               ...   
5117   SPBSS5IP                                     BSE SENSEX 50   
5118   SPBSSSIP                         BSE SmallCap Select Index   
5119   SPBSTLIP                                       BSE Telecom   
5120   SPBSUTIP                                     BSE Utilities   
5121   SPICBLOT                             BSE Liquid Rate Index   

                                            description         refreshed_at  \
0     End of Day prices and additional trading infor...  2023-09-01 14:24:43   
1     End o

In [6]:
#Calling the data for calculating SMA
data = quandl.get('BSE/SPBSE5S', start_date='2009-10-05', end_date='2019-10-05') #SPBSE5S represents BSE 500 Shariah,Historical BSE indexes for BSE 500 Shariah
print(data)

            Open  High   Low    Close
Date                                 
2013-04-30  None  None  None  1825.25
2013-05-03  None  None  None  1839.69
2013-05-06  None  None  None  1857.80
2013-05-07  None  None  None  1868.62
2013-05-08  None  None  None  1869.93
...          ...   ...   ...      ...
2019-09-27  None  None  None  4046.62
2019-09-30  None  None  None  4046.51
2019-10-01  None  None  None  4005.74
2019-10-03  None  None  None  3987.24
2019-10-04  None  None  None  3958.80

[1214 rows x 4 columns]


In [ ]:
#Function calculating the SMA 
# Load historical price data into a DataFrame
# Replace 'your_price_data.csv' with your own dataset
df = pd.read_csv('your_price_data.csv')

# Define the period and standard deviation multiplier
period = 20  # Lookback period for moving average and standard deviation
std_multiplier = 2  # Multiplier for standard deviation

# Calculate the rolling mean (middle band) and rolling standard deviation
df['Middle Band'] = df['Close'].rolling(window=period).mean()
df['Upper Band'] = df['Middle Band'] + (df['Close'].rolling(window=period).std() * std_multiplier)
df['Lower Band'] = df['Middle Band'] - (df['Close'].rolling(window=period).std() * std_multiplier)

In [ ]:
# Generate buy and sell signals
df['Signal'] = 0  # Initialize signal column

# Buy signal: Price crosses below the lower Bollinger Band
df.loc[df['Close'] < df['Lower Band'], 'Signal'] = 1

# Sell signal: Price crosses above the upper Bollinger Band
df.loc[df['Close'] > df['Upper Band'], 'Signal'] = -1


In [ ]:
# Plot Bollinger Bands and trading signals
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(df['Close'], label='Price', alpha=0.5)
plt.plot(df['Middle Band'], label='Middle Band', linestyle='--')
plt.plot(df['Upper Band'], label='Upper Band', linestyle='--')
plt.plot(df['Lower Band'], label='Lower Band', linestyle='--')

# Plot buy signals as green arrows
plt.scatter(df.index[df['Signal'] == 1], df['Close'][df['Signal'] == 1], marker='^', color='g', label='Buy Signal', alpha=1)

# Plot sell signals as red arrows
plt.scatter(df.index[df['Signal'] == -1], df['Close'][df['Signal'] == -1], marker='v', color='r', label='Sell Signal', alpha=1)

plt.title('Bollinger Bands Trading Strategy')
plt.legend()
plt.show()

In [ ]:
#Test it on the other chuck of time period 

In [ ]:
#Performance measurement